<!--Información del curso-->
<img align="left" style="padding-right:10px;" src="figuras/logo_ciencia_datos.png">

<center><h1 style="font-size:2em;color:#2467C0"> Pandas -Parte 7  </h1></center>

<center><h2 style="font-size:2em;color:#840700">  Agrupando datos  </h4></center>

<br>
<table>
<col width="550">
<col width="450">
<tr>
<td><img src="figuras/agrupando.png" align="left" style="width:500px"/></td>
<td>

* **Wes McKinney**, empezó a desarrollar Pandas en el año 2008 mientras trabajaba en *AQR Capital* [https://www.aqr.com/] por la necesidad que tenía de una herramienta flexible de alto rendimiento para realizar análisis cuantitativos en datos financieros. 
* Antes de dejar AQR convenció a la administración de la empresa de distribuir esta biblioteca bajo licencia de código abierto.
* **Pandas** es un acrónimo de **PANel DAta analysiS**
   
    
<br>
</td>
</tr>
</table>

# Librerías

Cargando las bibliotecas que necesitamos 


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Datos utilizados en esta Notebook

El archivo *planetas.csv* contiene datos sobre planetas que se han descubierto orbitando estrellas fuera de nuestro sistema solar. En este archivo, cada fila corresponde a un exoplaneta descubierto. Los atributos de cada exoplaneta (y por lo tanto las columnas del archivo) son:

* **método**  el método utilizado para descubrir el planeta.
* **número** el número total de planetas descubiertos orbitando la estrella anfitriona de este exoplaneta.
* **orbital_period** el período del planeta, su "año".
* **masa** la masa del exoplaneta.
* **distancia** la distancia de la estrella anfitriona del exoplaneta a la Tierra en años luz.
* **año** el año en que se descubrió el planeta.

In [4]:
# cargar el archivo datos/planets.csv en un dataframe llamado df_planetas
df_planetas = pd.read_csv('datos/planets.csv')
df_planetas

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10,77.40,2006
1,Radial Velocity,1,874.774000,2.21,56.95,2008
2,Radial Velocity,1,763.000000,2.60,19.84,2011
3,Radial Velocity,1,326.030000,19.40,110.62,2007
4,Radial Velocity,1,516.220000,10.50,119.47,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006
1031,Transit,1,2.615864,NaN,148.00,2007
1032,Transit,1,3.191524,NaN,174.00,2007
1033,Transit,1,4.125083,NaN,293.00,2008


In [5]:
#Numero total de filas y columnas
df_planetas.shape

(1035, 6)

In [6]:
#Caracteristicas de cada columna
df_planetas.describe()

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


Como se puede observar existen columnas que no tienen todos los datos (ver el parametro **count** que es diferente, solo cuenta cuando existe el dato)

#  Introducción

A menudo se require agrupar a través de alguna etiqueta en los *DataFrtames*: esto se implementa en la operación denominada ``groupby``. El nombre "group by" o "agrupar por"  proviene de un comando en el lenguaje de la base de datos **SQL**, pero quizás sea más esclarecedor pensar en los términos acuñados por primera vez por Hadley Wickham: dividir, aplicar, combinar.

En esta figura se ilustra un ejemplo canónico de esta operación *dividir-aplicar-combinar (split-apply-combine)*, donde "aplicar" es una *agregación de suma*:

<img align="left" width="550"  float= "none" align="middle" src="figuras/groupby.png">


Esto deja en claro lo que logra ``groupby``:

- El paso *split* o *dividir*, implica dividir y agrupar un *DataFrame* según la etiqueta especificada.
- El paso *apply* o *aplicar*, implica calcular alguna función, generalmente un agregado, transformación o filtrado, dentro de los grupos individuales.
- El paso  *combine* o *combinar*, fusiona los resultados de estas operaciones en un arreglo de salida.

Si bien esto ciertamente se podría hacer manualmente usando alguna combinación de los comandos de enmascaramiento, agregación y concatenación cubiertos en las lecciones anteriores, el poder de ``groupby`` es que abstrae estos pasos: el usuario no necesita pensar en *cómo* se realiza el cálculo, sino que piensa en la *operación como un todo*.

Como ejemplo concreto, echemos un vistazo al uso de **Pandas** para el cálculo que se muestra en este diagrama.
Comenzaremos creando el  *DataFrame*

In [7]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


La operación más básica de dividir-aplicar-combinar se puede calcular con el método ``groupby``, pasando el nombre de la columna de etiqueta deseada:

In [8]:
#groupby('key')
df.groupby('key')

Observe que lo que se retorna no es un conjunto de *DataFrames*, sino un objeto ``DataFrameGroupBy``. Este objeto es donde está la magia: puede pensar en él como una vista especial del DataFrame, que está preparado para profundizar en los grupos, pero no realiza ningún cálculo real hasta que se aplica la agregación. Este enfoque de "evaluación perezosa" significa que los agregados comunes se pueden implementar de manera muy eficiente de una manera casi transparente para el usuario.

Para producir un resultado, podemos aplicar un agregado a este objeto ``DataFrameGroupBy``, que realizará los pasos apropiados de aplicación/combinación para producir el resultado deseado:

In [9]:
#Sumando los elementos de las etiquetas que coinciden
#groupby('key').sum()
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


El método ``sum()``  es solo una posibilidad aquí; puede aplicar virtualmente cualquier función de agregación Pandas o NumPy común, así como virtualmente cualquier operación válida de *DataFrame*

In [10]:
#El elemento mayor 
#groupby('key').max()
df.groupby('key').max()

,data
key,
A,3
B,4
C,5


### El objeto GroupBy

El objeto ``GroupBy`` es una abstracción muy flexible. En muchos sentidos, puede simplemente tratarlo como si fuera una colección de *DataFrames* y hace las cosas difíciles sin que se percate de las operaciones y cálculos realizados. Veamos algunos ejemplos usando los datos de exoplanetas.

Quizás las operaciones más importantes que ofrece un ``GroupBy`` son el *agregado*, *filtrado*, *transformación* y *aplicación de un función*.

Discutiremos cada uno de estos con más detalle en las siguientes subsecciones, pero antes de eso, vamos a presentar algunas de las otras funciones que se pueden usar con la operación básica de ``GroupBy``.

#### Indexación de columnas

El objeto ``GroupBy`` admite la indexación de columnas de la misma manera que el *DataFrame* y devuelve un objeto ``GroupBy`` modificado.
Por ejemplo:

In [11]:
#groupby('method') 
df_planetas.groupby('method')

In [12]:
# method -> 'orbital_period' 
df_planetas.groupby('method')['orbital_period']

Aquí hemos seleccionado un grupo en particular del grupo *DataFrame* original por referencia a su nombre de columna. Al igual que con el objeto ``GroupBy``, no se realiza ningún cálculo hasta que llamamos a algún agregado en el objeto:

In [14]:
# method -> 'orbital_period' -> median()  
df_planetas.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

Si requerimos la lista ordenada alfabéticamente requerimos utilizar la función ``reset_index()``

In [15]:
#reset_index()
df_planetas.groupby('method')['orbital_period'].median().reset_index()

,method,orbital_period
0,Astrometry,631.180000
1,Eclipse Timing Variations,4343.500000
2,Imaging,27500.000000
3,Microlensing,3300.000000
4,Orbital Brightness Modulation,0.342887
5,Pulsar Timing,66.541900
6,Pulsation Timing Variations,1170.000000
7,Radial Velocity,360.200000
8,Transit,5.714932
9,Transit Timing Variations,57.011000


Ahora es posible aplicar el orden a la columna "orbital_period"

In [18]:
#ordenando
df_planetas.groupby('method')['orbital_period'].median().reset_index().sort_values('orbital_period', ascending=False)

,method,orbital_period
2,Imaging,27500.000000
1,Eclipse Timing Variations,4343.500000
3,Microlensing,3300.000000
6,Pulsation Timing Variations,1170.000000
0,Astrometry,631.180000
7,Radial Velocity,360.200000
5,Pulsar Timing,66.541900
9,Transit Timing Variations,57.011000
8,Transit,5.714932
4,Orbital Brightness Modulation,0.342887


Esto da una idea de la escala general de períodos orbitales (en días) a los que es sensible cada método. Podemos hacer lo mismo agrupando con lo el año.

In [19]:
# 'year' -> 'distance' -> median() 
df_planetas.groupby('year')['distance'].median()

year
1989     40.570
1992        NaN
1994        NaN
1995     15.360
1996     14.840
1997     17.430
1998     21.290
1999     29.760
2000     32.865
2001     33.650
2002     37.440
2003     35.955
2004     48.950
2005     35.870
2006     46.940
2007     80.580
2008     56.070
2009     52.830
2010     90.000
2011     87.870
2012    200.000
2013    132.000
2014    272.000
Name: distance, dtype: float64

Como era de esperarse se descubren planetas mas lejanos conforme va mejorando la tecnología a través de los años.

Para el propósito de las siguientes subsecciones, usaremos este *DataFrame*:

In [20]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


## a)  ``aggregate()`` - agregación


Puede tomar una cadena, una función o una lista de las mismas y calcular todos los agregados a la vez (puede hacer uso de las funciones de Numpy). Aquí hay un ejemplo rápido que combina todos estos:

In [21]:
# [min, np.median, max]
df.groupby('key').aggregate([np.min, np.median, np.max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

Otro patrón útil es pasar un diccionario de nombres de columna de mapeo a operaciones que se aplicarán en esa columna:

In [22]:
# {'data1': 'min','data2': 'max'}
df.groupby('key').aggregate({'data1': 'min','data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


Aplicando la operación similar a los datos de los exoplanetas 

In [23]:
# mostramos el dataframe df_planetas
df_planetas

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10,77.40,2006
1,Radial Velocity,1,874.774000,2.21,56.95,2008
2,Radial Velocity,1,763.000000,2.60,19.84,2011
3,Radial Velocity,1,326.030000,19.40,110.62,2007
4,Radial Velocity,1,516.220000,10.50,119.47,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006
1031,Transit,1,2.615864,NaN,148.00,2007
1032,Transit,1,3.191524,NaN,174.00,2007
1033,Transit,1,4.125083,NaN,293.00,2008


Agrupando por año y mostrando el "min", "median" y "max" en cada una de las otros parámetros cuantitativos (numéricos)

In [30]:
df_planetas.dtypes

method             object
number              int64
orbital_period    float64
mass              float64
distance          float64
year                int64
dtype: object

In [31]:
# 'year' -> [min, np.median, max]
df_planetas.groupby('year')[['number', 'orbital_period', 'mass', 'distance', 'year']].aggregate([min, np.median, max])

number            orbital_period                                 mass  \
        min median max            min      median            max       min   
year                                                                         
1989      1    1.0   1      83.888000   83.888000      83.888000  11.68000   
1992      3    3.0   3      25.262000   45.901950      66.541900       NaN   
1994      3    3.0   3      98.211400   98.211400      98.211400       NaN   
1995      1    1.0   1       4.230785    4.230785       4.230785   0.47200   
1996      1    2.0   5       3.313500   65.669700    1078.000000   0.68760   
1997      1    1.0   1      39.845000   39.845000      39.845000   1.04000   
1998      1    2.0   4       3.097000   58.112890     442.100000   0.52000   
1999      1    1.0   4       3.510000  256.780000    3810.000000   0.42000   
2000      1    1.0   4       3.024000  179.665000    2502.000000   0.24900   
2001      1    1.0   3       6.276000  462.790000    2391.000000   0.54000   
2002      1    1.0   5       1.211919  505.549000    4909.000000   0.11500   
2003      1    1.0   3       1.328300  441.470000   36525.000000   0.22900   
2004      1    1.0   5       1.432475   35.103500    4205.800000   0.08700   
2005      1    1.0   4       1.937780   68.630000    4270.000000   0.00600   
2006      1    1.0   4       1.796000  233.150000  730000.000000   0.03300   
2007      1    1.0   5       1.306186  311.665000    1333.000000   0.03500   
2008      1    1.0   6       1.091423  226.930000  318280.000000   0.01260   
2009      1    1.0   3       0.788840  464.700000   17337.500000   0.01600   
2010      1    1.0   6       1.212884   16.278350   18000.000000   0.01300   
2011      1    2.0   6       0.090706   20.219180   40000.000000   0.00755   
2012      1    2.0   6       0.742960   11.738714    6005.000000   0.00360   
2013      1    2.0   7       0.355000   24.806000    4100.000000   0.00800   
2014      1    2.0   5       0.669310   12.815900    3827.000000   0.34000   

                      distance                    year                
       median     max      min   median      max   min  median   max  
year                                                                  
1989  11.6800  11.680    40.57   40.570    40.57  1989  1989.0  1989  
1992      NaN     NaN      NaN      NaN      NaN  1992  1992.0  1992  
1994      NaN     NaN      NaN      NaN      NaN  1994  1994.0  1994  
1995   0.4720   0.472    15.36   15.360    15.36  1995  1995.0  1995  
1996   1.6650   3.900    12.53   14.840    21.41  1996  1996.0  1996  
1997   1.0400   1.040    17.43   17.430    17.43  1997  1997.0  1997  
1998   1.3900   8.020     4.70   21.290    47.92  1998  1998.0  1998  
1999   1.9855   8.440    10.91   29.760    59.03  1999  1999.0  1999  
2000   1.4150  18.100     3.22   32.865    80.00  2000  2000.0  2000  
2001   2.0700  10.350    14.08   33.650    77.82  2001  2001.0  2001  
2002   2.2500  17.400    12.53   37.440   121.36  2002  2002.0  2002  
2003   1.8250  18.370    11.11   35.955   133.16  2003  2003.0  2003  
2004   1.7900   3.690    10.23   48.950  2500.00  2004  2004.0  2004  
2005   0.5610  14.000     4.70   35.870   300.30  2005  2005.0  2005  
2006   0.9435   7.600     8.77   46.940  8500.00  2006  2006.0  2006  
2007   2.2150  19.800     4.54   80.580  2000.00  2007  2007.0  2007  
2008   1.9900  25.000     4.94   56.070   680.00  2008  2008.0  2008  
2009   1.7800  21.420     8.52   52.830   550.00  2009  2009.0  2009  
2010   0.6000  13.000     4.70   90.000  2700.00  2010  2010.0  2010  
2011   0.1200  13.650     6.06   87.870  2250.00  2011  2011.0  2011  
2012   2.3700  11.100     1.35  200.000  7720.00  2012  2012.0  2012  
2013   0.9665  20.600     6.80  132.000  7560.00  2013  2013.0  2013  
2014   0.4800   2.580    20.48  272.000  1056.00  2014  2014.0  2014

Nos fijamos ahora en la columna de “year” y en las variaciones de los valores de la columna “mass”

In [33]:
# 'year' -> 'mass' -> aggregate([min, np.median, max])
df_planetas.groupby('year')['mass'].aggregate([min, np.median, max])

,min,median,max
year,,,
1989,11.68000,11.6800,11.680
1992,NaN,NaN,NaN
1994,NaN,NaN,NaN
1995,0.47200,0.4720,0.472
1996,0.68760,1.6650,3.900
1997,1.04000,1.0400,1.040
1998,0.52000,1.3900,8.020
1999,0.42000,1.9855,8.440
2000,0.24900,1.4150,18.100


Podemos ordenar los datos de la columna "median" de mayor a menor

In [34]:
# 'year' -> 'mass' -> aggregate([np.min, np.median, np.max])
df_planetas.groupby('year')['mass'].aggregate([min, np.median, max]).sort_values('median', ascending=False)

,min,median,max
year,,,
1989,11.68000,11.6800,11.680
2012,0.00360,2.3700,11.100
2002,0.11500,2.2500,17.400
2007,0.03500,2.2150,19.800
2001,0.54000,2.0700,10.350
2008,0.01260,1.9900,25.000
1999,0.42000,1.9855,8.440
2003,0.22900,1.8250,18.370
2004,0.08700,1.7900,3.690


## b)  apply ()  - Aplicación

El método  ``apply()`` le permite aplicar una función arbitraria a los resultados del grupo.
La función debe tomar un *DataFrame* y devolver un objeto Pandas  o un escalar; la operación de combinación se adaptará al tipo de salida devuelta.

Por ejemplo, aquí hay una "aplicación" que normaliza la primera columna por la suma de la segunda:

In [35]:
def norm_by_data2(x):
    #print("x['data2'].sum() )
    x['data1'] /= x['data2'].sum()
    return x

In [36]:
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


In [37]:
#groupby('key')['data2'].sum()
df.groupby('key')['data2'].sum()

key
A     8
B     7
C    12
Name: data2, dtype: int64

In [38]:
# Todos los elementos de data1-subgrupoA se dividirán entre 8
# Todos los elementos de data1-subgrupoB se dividirán entre 7
# Todos los elementos de data1-subgrupoC se dividirán entre 12
# groupby('key', group_keys=False).apply(norm_by_data2)
df.groupby('key', group_keys=False).apply(norm_by_data2)

,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3
3,A,0.375000,3
4,B,0.571429,7
5,C,0.416667,9
